## Complete Code

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import MetaData
from sqlalchemy import insert
from sqlalchemy import text
from sqlalchemy import update
from sqlalchemy import exc


def make_tb(engine, tb_name, columns):
    meta = MetaData()
    # create a table object called students
    tb = Table(
        tb_name,
        meta, # the metadata obj to hold this table info
        *columns    
    )

    meta.create_all(engine)
    return tb

    
def insert_rows(tb, rows) -> bool:
    result = None
    with engine.connect() as conn:
        ins = tb.insert().values(rows)
        conn.commit()
        return True
    return False

    
def upsert_all(tb, rows) -> bool:
    """On insert conflict do update."""
    upsert_success = False
    conn = engine.connect()
    for row in rows:
        upsert_one(conn, tb, row)

    if upsert_success:            
        conn.commit()
        
    conn.close()
    return result

def upsert_one(conn, tb, row):
    upsert_success = False
    try:
        ins = tb.insert().values(**row)
        conn.execute(ins)
        upsert_success = True
        print("Insert succeeded")
    except (exc.IntegrityError, Exception) as err:
        print("Insert Error:", err)
        try:
            upd = tb.update().where(tb.c.id==row['id']).values(**row)
            conn.execute(upd)
            upsert_success = True
            print("Update succeeded")
        except Exception as err:
            print("Update Error:", err)
            upsert_success = False
            
    return upsert_success




    
# run code

engine = create_engine('sqlite:///test.db', echo=True)

columns = [
    Column('id', Integer, primary_key=True), 
    Column('name', String), 
    Column('age', String), 
]


tb_name = "test_table_4"

tb = make_tb(engine, tb_name, columns)

In [ ]:
# Design ideas
"""
# database insert & update
1. primary key = project_name + rule_name + date(day)
2. Create a dev_data_monitor.db when using "data-monitor init"
3. The entry_point provides flag load_to_db=True, env=dev
4. When record is sent to db, it upserts the records (update on conflict)

# retrieve data
1. A utility class (using db client, e.g., TestDbClient)
  test_db_client = SQLDBClient(host="sqlite")
"""

In [ ]:
rows = [
    {"id":1, "name": "Hey", "age": 2},
    {"id":2, "name": "Kyle", "age": 10},
]

with engine.connect() as conn:
    if upsert_one(conn, tb, rows[0]):
        conn.commit()

In [ ]:
conn =  engine.connect()

In [ ]:
qry = """
select * from test_table_4
"""

cursor = conn.execute(text(qry))

In [ ]:
data = cursor.fetchall()

In [ ]:
data

## Use ORM (maybe better way for insert?)

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, declarative_base

engine = create_engine('sqlite:///data_monitor.db', echo = True)

Base = declarative_base()


class DataMonitor(Base):
    __tablename__ = "data_monitor"
    id = Column(String, primary_key=True)
    field_name = Column(String)
    value = Column(Integer)
    rule_name = Column(String)
    
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
field_name = "shares"
value = 10
rule_name = "shares_check"
primary_key = "|".join([field_name, rule_name])
new_row = DataMonitor(id=primary_key, field_name=field_name, value=value, rule_name=rule_name)
session.add(new_row)
session.commit()

In [ ]:
session.get(DataMonitor, "shares|shares_check").field_name

In [ ]:
session.close()

In [ ]:
rows = [
    {"name":"Shin", "age":20},
    {"name":"Shin", "age":20},
    {"name":"Shin", "age":20},
]

for row in rows:
    session.add(DataMonitor(**row))
session.commit()

In [ ]:
session.commit()

In [ ]:
session.get(DataMonitor, 3).age

# Use ORM to get a row as an object

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, declarative_base

engine = create_engine('sqlite:///test.db', echo = True)

Base = declarative_base()


class TestTable4(Base):
    __tablename__ = "test_table_4"
    id = Column(Integer, primary_key=True)
    name = Column('name', String) 
    age = Column('age', String)
    
Session = sessionmaker(bind=engine)
session = Session()
session.get(TestTable4, 10).name

In [ ]:
session.close()

In [ ]:
from jinja2 import environment
import re

In [ ]:
string = """
//some stuff
set @x = {{ x | default (20)}}
aefefwefe

set @y = {{y|default(30)}}
"""
vars = ["x", "y"]
var_str = "|".join([v for v in vars])
reg = f"({var_str})\s*\|\s*default\s*\((\w*)\)"

In [ ]:
reg

In [ ]:
itr = re.findall(reg, string, re.MULTILINE)

In [ ]:
for i in itr:
    print(i)

In [ ]:
class MyClass:
    def __init__(self):
        pass
    
    def __call__(self, a:str):
        return a


In [ ]:
m = MyClass()

In [ ]:
def my_func(a:str, b:int):
    return a

In [ ]:
import singleton

In [ ]:
class Engine:
    
    def __new__(cls, *args):
        if not hasattr(cls, "instance"):
            print("create instance", cls)
            cls.instance = super().__new__(cls)
        return cls.instance
    
    def __init__(self, a, b):
        print("instance created")
        self.a = a
        self.b = b
    
    

In [ ]:
a = Engine(1, 2)

In [ ]:
b = Engine(3, 4)

In [ ]:
a.a

In [ ]:
b.a

In [ ]:
b.number

In [2]:
vars()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['', 'vars', 'vars()'],
 '_oh': {1: <function vars>},
 '_dh': [PosixPath('/Users/hsin/git/sqlalchemy')],
 'In': ['', 'vars', 'vars()'],
 'Out': {1: <function vars>},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x10be4f6a0>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x10be4feb0>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x10be4feb0>,
 'open': <function io.open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)>,
 '_': <function vars>,
 '__': '',
 '___': '',
 '_i': 'vars',
 '_ii': '',
 '_iii': '',
 '_i1': 'vars',
 '_1': <function vars>,
 '_i2': 'vars()'}

In [3]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['', 'vars', 'vars()', 'globals()'],
 '_oh': {1: <function vars>, 2: {...}},
 '_dh': [PosixPath('/Users/hsin/git/sqlalchemy')],
 'In': ['', 'vars', 'vars()', 'globals()'],
 'Out': {1: <function vars>, 2: {...}},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x10be4f6a0>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x10be4feb0>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x10be4feb0>,
 'open': <function io.open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)>,
 '_': {...},
 '__': <function vars>,
 '___': '',
 '_i': 'vars()',
 '_ii': 'vars',
 '_iii': '',
 '_i1': 'vars',
 '_1': <function var